# Python developer

## Тестовое задание

Вы – аналитик больших данных, которому Глава города Нижнего Новгорода поручил **провести исследование о состоянии туризма в городе**.

Все необходимые данные собирает третья сторона, которая может автоматизировано делиться с Вами CSV выгрузками.

**Ваша задача** – настроить автоматическую обработку и загрузку данных, поступающих от третей стороны, разработать интерфейс передачи данных во внутреннюю ERP систему (REST API) и, на основе полученной информации подготовить аналитику, которая позволит Главе города принимать управленческие решения.

## Основные этапы

1. Создать базу данных PostgreSQL на Yandex.Cloud. (Зарегистрируйте новую учетную запись с бесплатным пробным периодом и разверните БД в сервисе Managed Services for PostgreSQL);
2. С **помощью Python скрипта** предобработать тестовую выгрузку и загрузить ее в БД;
3. Основные ключевые показатели, которые Вы придумаете и подсчитаете, собрать в JSON файл и **настроить локальный RestAPI с использованием Flask**, для получения этого файла с помощью GET/POST запроса;

## Вопросы, на которые нужно дать ответ в аналитике

1. Сколько туристов посетило Нижний Новгород за весь диапазон дат, представленных в выборке;
2. Сколько туристов посещало Нижний Новгород каждый месяц и за произвольный диапазон времени;
3. Как представлено территориальное распределение туристов (откуда они приехали);
4. Как представлено демографическое распределение туристов;
5. Под какую категорию туристов выгоднее всего планировать мероприятия и события (в денежном и количественном эквиваленте);
6. Как выглядит профиль среднестатистического туриста, посетившего город.

In [35]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from datetime import datetime

## 1. Сколько туристов посетило Нижний Новгород за весь диапазон дат, представленных в выборке

In [36]:
url = "http://localhost:5000/api/question/1"

try:
    response = requests.get(url)
    data = response.json()
    
    total = data['answer']['total_visitors']
    formatted = data['answer']['total_visitors_formatted']
    
    print(f"\nОбщее количество туристов: {formatted}")
    
except Exception as e:
    print(f"Ошибка: {e}")
    print("Убедитесь, что API запущен (analytics.py)")


Общее количество туристов: 2,511,801 человек


## 2. Сколько туристов посещало Нижний Новгород каждый месяц и за произвольный диапазон времени?

### За каждый месяц:

In [37]:
url = "http://localhost:5000/api/question/2"

try:
    response = requests.get(url)
    data = response.json()
    
    for month in data['answer']['months']:
        print(f" {month['month']}:")
        print(f"    Туристов: {month['visitors']} чел.")
      
except Exception as e:
    print(f"Ошибка: {e}")
    print("Убедитесь, что API запущен (analytics.py)")

 2021-01:
    Туристов: 357146 чел.
 2021-02:
    Туристов: 330578 чел.
 2021-03:
    Туристов: 399004 чел.
 2021-04:
    Туристов: 386198 чел.
 2021-05:
    Туристов: 515151 чел.
 2021-06:
    Туристов: 523724 чел.


### За произвольный диапазон:

In [38]:
start_date = "2021-01-01"
end_date = "2021-02-01"

url = f"http://localhost:5000/api/question/2?start_date={start_date}&end_date={end_date}"

try:
    response = requests.get(url)
    data = response.json()

    print(f"Туристы за период: {start_date} - {end_date}")
    print("-"*50)
    
    for month in data['answer']['months']:
        print(f" {month['month']}:")
        print(f"    Туристов: {int(month['visitors']):,} чел")
        print(f"    Потрачено: {int(month['spent']):,} руб")
        print()
    
except Exception as e:
    print(f"Ошибка: {e}")
    print("Убедитесь, что API запущен (analytics.py)")

Туристы за период: 2021-01-01 - 2021-02-01
--------------------------------------------------
 2021-01:
    Туристов: 357,146 чел
    Потрачено: 2,995,479,000 руб

 2021-02:
    Туристов: 12,567 чел
    Потрачено: 131,175,000 руб



## 3. Как представлено территориальное распределение туристов (откуда они приехали)?

In [39]:
url = "http://localhost:5000/api/question/3"

try:
    response = requests.get(url)
    data = response.json()

    print("\nТоп-10 стран:")
    print("-" * 80)
    
    countries = data['answer']['countries']
    sorted_countries = sorted(countries.items(), key=lambda x: x[1], reverse=True)
    
    for country, count in sorted_countries[:10]:
        print(f"{country:25} : {count:6,} туристов")
    
    print("\nТоп-10 регионов России:")
    print("-" * 80)
    
    regions = data['answer']['regions']
    russia_regions = [r for r in regions if r['country'] == 'Россия']
    
    for region in russia_regions[:10]:
        visitors = f"{region['visitors']:,}".replace(',', ' ')
        print(f"{region['region']:30} : {visitors:>10} туристов ({region['percent']}%)")
    
except Exception as e:
    print(f"Ошибка: {e}")
    print("Убедитесь, что API запущен (analytics.py)")


Топ-10 стран:
--------------------------------------------------------------------------------
Россия                    : 2,499,576 туристов
Белоруссия                :  1,151 туристов
Таджикистан               :  1,093 туристов
Эстония                   :    832 туристов
Китай (Китайская Народная Республика) :    784 туристов
Узбекистан                :    675 туристов
Турция                    :    644 туристов
Украина                   :    600 туристов
Германия                  :    570 туристов
Казахстан                 :    507 туристов

Топ-10 регионов России:
--------------------------------------------------------------------------------
Нижегородская область          :    255 135 туристов (10.16%)
Нижегородская область          :    199 611 туристов (7.95%)
г. Москва                      :    189 416 туристов (7.54%)
Нижегородская область          :    186 256 туристов (7.42%)
Нижегородская область          :    184 665 туристов (7.35%)
Московская область             :    1

## 4. Как представлено демографическое распределение туристов?

In [40]:
url = "http://localhost:5000/api/question/4"

try:
    response = requests.get(url)
    data = response.json()
    
    print("\n Возраст:")
    for age in data['answer']['ages']:
        print(f"  {age['group']:15} : {age['visitors']:>6,} чел. ({age['percent']}%)")
    
    print("\nПол:")
    for g in data['answer']['genders']:
        print(f"  {g['gender']:10} : {g['visitors']:>6,} чел.")
        
except Exception as e:
    print(f"Ошибка: {e}")


 Возраст:
  до 18 лет       : 82,178 чел. (3.29%)
  от 18 до 24 лет : 265,261 чел. (10.63%)
  от 25 до 34 лет : 556,832 чел. (22.31%)
  от 35 до 44 лет : 446,655 чел. (17.9%)
  от 45 до 54 лет : 397,212 чел. (15.91%)
  от 55 до 63 лет : 386,117 чел. (15.47%)
  свыше 64 лет    : 361,638 чел. (14.49%)

Пол:
  женский    : 1,309,479 чел.
  мужской    : 1,186,414 чел.


## 5. Под какую категорию туристов выгоднее всего планировать мероприятия и события (в денежном и количественном эквиваленте) ?

In [41]:

url = "http://localhost:5000/api/question/5"

try:
    response = requests.get(url)
    data = response.json()

    best = data['answer']['best']
    print("\nСамая выгодная категория туристов:")
    print("-" * 80)
    print(f"   Возраст: {best['age']}")
    print(f"   Доход: {best['income']}")
    print(f"   Траты на человека: {best['spent_person']} руб.")
    print(f"   Всего туристов: {best['visitors']} чел.")
    print(f"   Средняя продолжительность: {best['days']} дней")
    
    print("\nТоп-5 по тратам на человека:")
    print("-" * 80)
    print(f"{'Возраст':15} {'Доход':25} {'Траты/чел':>15} {'Кол-во':>10}")
    print("-" * 80)
    
    for cat in data['answer']['age_income'][:5]:
        print(f"{cat['age']:15} {cat['income']:25} {cat['spent_person']:>15} {cat['visitors']:>10}")
    
except Exception as e:
    print(f"Ошибка: {e}")
    print("Убедитесь, что API запущен (analytics.py)")


Самая выгодная категория туристов:
--------------------------------------------------------------------------------
   Возраст: от 55 до 63 лет
   Доход: свыше 100 тыс. руб.
   Траты на человека: 33318 руб.
   Всего туристов: 5914 чел.
   Средняя продолжительность: 5.2 дней

Топ-5 по тратам на человека:
--------------------------------------------------------------------------------
Возраст         Доход                           Траты/чел     Кол-во
--------------------------------------------------------------------------------
от 55 до 63 лет свыше 100 тыс. руб.                 33318       5914
свыше 64 лет    свыше 100 тыс. руб.                 32501       7290
от 35 до 44 лет свыше 100 тыс. руб.                 31900      10446
от 25 до 34 лет свыше 100 тыс. руб.                 31346      14857
от 45 до 54 лет свыше 100 тыс. руб.                 31294       6078


## 6. Как выглядит профиль среднестатистического туриста, посетившего город?

In [42]:
url = "http://localhost:5000/api/question/6"

try:
    response = requests.get(url)
    data = response.json()

    # Текстовое описание
    # print(f"\n {data['answer']['text']}")

    cat = data['answer']['categories']
    num = data['answer']['numbers']
    
    print(f"- Возраст: {cat['age']}")
    print(f"- Пол: {cat['gender']}")
    print(f"- Доход: {cat['income']}")
    print(f"- Цель поездки: {cat['goal']}")
    print(f"- Тип поездки: {cat['trip']}")
    print(f"- Откуда: {cat['region']} ({cat['city']})")
    print(f"- Дней пребывания: {num['days']} (в среднем)")
    print(f"- Размер группы: {num['group']} чел.")
    print(f"- Траты за поездку: {num['trip']} руб.")
    print(f"- Траты на человека: {num['person']} руб.")
    
except Exception as e:
    print(f"Ошибка: {e}")
    print("Убедитесь, что API запущен (analytics.py)")

- Возраст: от 25 до 34 лет
- Пол: женский
- Доход: до 30 тыс. руб.
- Цель поездки: Посещение друзей и родственников
- Тип поездки: Поездка внутри страны
- Откуда: Нижегородская область (Кстовский муниципальный район)
- Дней пребывания: 5.6 (в среднем)
- Размер группы: 5.2 чел.
- Траты за поездку: 49295 руб.
- Траты на человека: 10680 руб.
